In [1]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 771 kB 4.9 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=2626466 sha256=9c376e3fd70c79708243a0ea1697bf0665e291ba1e78ff6988eef49d32c33d64
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [2]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate, GridSearchCV, KFold
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from surprise import SVDpp, accuracy
from surprise import SVD, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering

In [3]:
import pickle
from sklearn.preprocessing import MinMaxScaler


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# df = pd.read_csv('/content/drive/Othercomputers/My Laptop/OneDrive/CMPE 256/Project/train_triplets_large.txt', sep='\t', index_col=None, names = ['User','Song', 'Count'])
df = pd.read_csv('/content/drive/MyDrive/Music Recommendation/train_triplets.txt', sep='\t', index_col=None, names = ['User','Song', 'Count'])
df.head()

,User,Song,Count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [ ]:
df[df['Count']==0]

NameError: ignored

In [ ]:
df.count()

User     12280732
Song     12280732
Count    12280732
dtype: int64

In [ ]:
# from plotly.offline import init_notebook_mode, plot, iplot
# import plotly.graph_objs as go
# init_notebook_mode(connected=True)

# data = df['Count'].value_counts().sort_index(ascending=False)
# trace = go.Bar(x = data.index,
#                text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
#                textposition = 'auto',
#                textfont = dict(color = '#000000'),
#                y = data.values,
#                )
# # Create layout
# layout = dict(title = 'Distribution Of {} book-ratings'.format(df.shape[0]),
#               xaxis = dict(title = 'Counts'),
#               yaxis = dict(title = 'Occurences'))
# # Create plot
# fig = go.Figure(data=[trace], layout=layout)
# iplot(fig)

In [ ]:
df['Count'] = df["Count"].astype(np.int64)

In [ ]:
a = df["Count"].value_counts().sort_index()

In [ ]:
index_list = a.index.values.tolist()

In [ ]:
listindex = [i for i in index_list if i >= 1000 or i<=2]

In [ ]:
df = df[~df['Count'].isin(listindex)]

In [ ]:
min_count = min(df["Count"])
max_count = max(df["Count"])

In [ ]:
min_count

3

In [ ]:
users = pd.read_csv('/content/drive/MyDrive/Music Recommendation/train_users.csv', sep=',', index_col=None, names = ['User','Count'], skiprows=1)
# users = pd.read_csv('/content/drive/MyDrive/256 Project/Music Recommendation/train_users.csv', sep=',', index_col=None, names = ['User','Count'], skiprows=1)
users

,User,Count
0,10c3da065896b3f9c10845aa62a788a56590fd84,166
1,6e8ac84fa420ef37839559491f664644bc7349b8,103
2,cd3b3a18ccc912a9d87dec984fe8a46fa6dd05ab,13
3,8b6d7a53a4311f9ecf43c09a99e261f378035b87,61
4,ecd0416d0e4f292dc0570f5e6b87521e11622d68,84
...,...,...
815449,16ae0e0818b4c3b99134380464b858b6b14b5ada,61
815450,1fa88d828dc104feb4e6bb11f360e6601178b2a4,34
815451,6b53f11ebc58dc591dc6c88fbee3c220386d1b80,16
815452,297d20c3f24a11c7264ae6249fb0bf5c303c5120,60


In [ ]:
# users.drop(index=users.index[0], 
#         axis=0, 
#         inplace=True)

In [ ]:
users

,User,Count
0,10c3da065896b3f9c10845aa62a788a56590fd84,166
1,6e8ac84fa420ef37839559491f664644bc7349b8,103
2,cd3b3a18ccc912a9d87dec984fe8a46fa6dd05ab,13
3,8b6d7a53a4311f9ecf43c09a99e261f378035b87,61
4,ecd0416d0e4f292dc0570f5e6b87521e11622d68,84
...,...,...
815449,16ae0e0818b4c3b99134380464b858b6b14b5ada,61
815450,1fa88d828dc104feb4e6bb11f360e6601178b2a4,34
815451,6b53f11ebc58dc591dc6c88fbee3c220386d1b80,16
815452,297d20c3f24a11c7264ae6249fb0bf5c303c5120,60


In [ ]:
users_list = users.User.values.tolist()
# users_list

In [ ]:
new_df = df[df['User'].isin(users_list)]
new_df

,User,Song,Count
13,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5
33,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOIZAZL12A6701C53B,5
37,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOKRIMP12A6D4F5DA3,5
45,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOMGIYR12AB0187973,6
48,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOMZWUW12A8C1400BC,6
...,...,...,...
48373544,cf8289419383259189afe6bb50c5115fd84f1064,SOYSPLQ12AB0185D3F,7
48373545,cf8289419383259189afe6bb50c5115fd84f1064,SOZCDWG12A6D4F81E1,6
48373564,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOOFYTN12A6D4F9B35,4
48373574,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOSQJWM12A6D4F79E0,6


In [ ]:
# new_df.to_csv('/content/drive/MyDrive/256 Project/Music Recommendation/select_users.csv')

In [5]:
new_df = pd.read_csv('/content/drive/MyDrive/Music Recommendation/select_users.csv', sep=',', index_col=None, names = ['User','Song', 'Count'], skiprows=1)

In [6]:
new_df

,User,Song,Count
13,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5
33,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOIZAZL12A6701C53B,5
37,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOKRIMP12A6D4F5DA3,5
45,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOMGIYR12AB0187973,6
48,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOMZWUW12A8C1400BC,6
...,...,...,...
48373544,cf8289419383259189afe6bb50c5115fd84f1064,SOYSPLQ12AB0185D3F,7
48373545,cf8289419383259189afe6bb50c5115fd84f1064,SOZCDWG12A6D4F81E1,6
48373564,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOOFYTN12A6D4F9B35,4
48373574,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOSQJWM12A6D4F79E0,6


In [7]:
from numba import njit



In [8]:
@njit
def cut(arr):
  bins = np.empty(arr.shape[0])
  for idx, x in enumerate(arr):
    if (x > 0) & (x <= 1):
        bins[idx] = 0
    elif (x > 1) & (x < 5):
        bins[idx] = 1
    elif (x >= 5) & (x < 10):
        bins[idx] = 2
    elif (x >= 10) & (x < 20):
        bins[idx] = 3
    elif (x >= 20) & (x < 30):
        bins[idx] = 4
    else:
        bins[idx] = 5
  return bins

In [9]:
temp = cut(new_df['Count'].to_numpy())
new_df['Binned_Count'] = temp.astype(np.int32)

In [10]:
new_df


,User,Song,Count,Binned_Count
13,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5,2
33,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOIZAZL12A6701C53B,5,2
37,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOKRIMP12A6D4F5DA3,5,2
45,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOMGIYR12AB0187973,6,2
48,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOMZWUW12A8C1400BC,6,2
...,...,...,...,...
48373544,cf8289419383259189afe6bb50c5115fd84f1064,SOYSPLQ12AB0185D3F,7,2
48373545,cf8289419383259189afe6bb50c5115fd84f1064,SOZCDWG12A6D4F81E1,6,2
48373564,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOOFYTN12A6D4F9B35,4,1
48373574,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOSQJWM12A6D4F79E0,6,2


In [ ]:
cols = ['Count']
scaler = MinMaxScaler(feature_range = (1,100))
scaled_data = scaler.fit_transform(new_df[cols])

In [ ]:
x = scaled_data.tolist()

In [ ]:
# new_df['Count'] = x

In [ ]:
# new_df

,User,Song,Count
13,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,[0.0020161290322580645]
33,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOIZAZL12A6701C53B,[0.0020161290322580645]
37,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOKRIMP12A6D4F5DA3,[0.0020161290322580645]
45,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOMGIYR12AB0187973,[0.0030241935483870967]
48,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOMZWUW12A8C1400BC,[0.0030241935483870967]
...,...,...,...
48373544,cf8289419383259189afe6bb50c5115fd84f1064,SOYSPLQ12AB0185D3F,[0.004032258064516129]
48373545,cf8289419383259189afe6bb50c5115fd84f1064,SOZCDWG12A6D4F81E1,[0.0030241935483870967]
48373564,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOOFYTN12A6D4F9B35,[0.0010080645161290322]
48373574,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOSQJWM12A6D4F79E0,[0.0030241935483870967]


In [ ]:
x = scaler.inverse_transform(scaled_data).astype(np.int64)
x

array([[5.],
       [5.],
       [5.],
       ...,
       [4.],
       [6.],
       [3.]])

In [ ]:
new_df['User'] = str(new_df['User'])

<ipython-input-74-5fe795f13820>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['User'] = str(new_df['User'])


In [ ]:
new_df.dtypes

User     object
Song     object
Count     int64
dtype: object

In [11]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(new_df[['User', 'Song', 'Binned_Count']], reader)

In [34]:
svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8729  0.8719  0.8717  0.8721  0.0005  
MAE (testset)     0.6710  0.6706  0.6702  0.6706  0.0003  
Fit time          64.90   67.88   70.40   67.73   2.25    
Test time         49.90   47.01   47.99   48.30   1.20    


{'test_rmse': array([0.87285816, 0.87190261, 0.87165392]),
 'test_mae': array([0.67102364, 0.67059236, 0.67021546]),
 'fit_time': (64.90143418312073, 67.87660026550293, 70.3978660106659),
 'test_time': (49.9041690826416, 47.008610248565674, 47.98935580253601)}

In [35]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [36]:
with open('/content/drive/MyDrive/Music Recommendation/svd_model_bins_new.pkl', 'wb') as f:
  pickle.dump(svd, f)

In [ ]:
test_users = pd.read_csv('/content/drive/MyDrive/Music Recommendation/test_users.csv', sep=',', index_col=None, names = ['User','Count'], skiprows=1)


In [26]:
test_users_list = test_users.User.values.tolist()

In [27]:
test_df = df[df['User'].isin(test_users_list)]

In [28]:
test_df

,User,Song,Count
104,85c1f87fea955d09b4bec2e36aee110927aedf9a,SOACWYB12AF729E581,2
105,85c1f87fea955d09b4bec2e36aee110927aedf9a,SOAUSXX12A8C136188,1
106,85c1f87fea955d09b4bec2e36aee110927aedf9a,SOBVAHM12A8C13C4CB,1
107,85c1f87fea955d09b4bec2e36aee110927aedf9a,SODJTHN12AF72A8FCD,2
108,85c1f87fea955d09b4bec2e36aee110927aedf9a,SOELXOV12A8C13616B,2
...,...,...,...
48373184,5e204405f0bd87efc0a3b407b87bf117ef527232,SOTIBMD12A67020A45,1
48373185,5e204405f0bd87efc0a3b407b87bf117ef527232,SOUANBT12A6D4F9129,1
48373186,5e204405f0bd87efc0a3b407b87bf117ef527232,SOUAUIZ12A8C140C20,1
48373187,5e204405f0bd87efc0a3b407b87bf117ef527232,SOYTZBN12AB0187A0C,1


In [23]:
test_df.to_csv('/content/drive/MyDrive/Music Recommendation/test_users_final.csv')

In [37]:
test_users = pd.read_csv('/content/drive/MyDrive/Music Recommendation/test_users_final.csv', sep=',', index_col=None, names = ['User','Song', 'Count'], skiprows=1)


In [38]:
recs = test_df[['User','Song']]

In [39]:
values = []
for index, row in recs.head(2000).iterrows():
  pred = svd.predict(uid=row['User'], iid=row['Song'])
  values.append(round(pred.est, 2))

In [40]:
values

[1.69,
 1.78,
 1.86,
 1.67,
 1.67,
 1.83,
 1.74,
 1.76,
 1.91,
 1.8,
 1.65,
 1.8,
 1.8,
 1.82,
 1.71,
 1.84,
 1.75,
 1.86,
 1.8,
 1.67,
 1.75,
 1.85,
 1.77,
 2.19,
 1.77,
 1.89,
 1.85,
 2.26,
 1.96,
 2.17,
 2.05,
 1.79,
 2.1,
 1.82,
 1.61,
 1.88,
 2.17,
 1.75,
 1.63,
 1.81,
 2.26,
 1.91,
 1.67,
 2.26,
 1.67,
 2.11,
 2.12,
 2.13,
 1.66,
 2.02,
 1.73,
 1.56,
 1.78,
 2.0,
 1.73,
 1.69,
 1.79,
 1.77,
 1.7,
 2.06,
 2.17,
 1.67,
 2.49,
 1.77,
 1.62,
 1.72,
 1.69,
 2.22,
 1.64,
 1.64,
 1.8,
 1.72,
 2.14,
 2.1,
 1.49,
 1.52,
 1.61,
 1.62,
 2.29,
 1.44,
 1.6,
 1.68,
 1.71,
 1.95,
 1.95,
 2.12,
 1.92,
 1.55,
 2.02,
 1.55,
 1.66,
 1.65,
 1.91,
 1.78,
 1.71,
 1.54,
 1.55,
 1.5,
 1.72,
 1.53,
 1.78,
 1.81,
 1.64,
 1.49,
 1.62,
 1.72,
 1.93,
 1.73,
 1.63,
 1.47,
 1.56,
 1.69,
 1.64,
 1.64,
 1.89,
 1.92,
 1.68,
 1.67,
 1.6,
 1.73,
 1.61,
 1.55,
 1.65,
 1.62,
 2.25,
 1.62,
 1.78,
 1.49,
 1.6,
 1.74,
 1.92,
 1.55,
 1.86,
 1.72,
 1.59,
 1.69,
 1.91,
 1.97,
 1.89,
 1.73,
 1.48,
 1.82,
 1.65,
 1.78,
 1.46

In [ ]:
# recs['Count'] = values

In [ ]:
recs.Count.value_counts